# Unit 2 Deep Dive into Agent Results and Chaining

## Extracting Input and Final Output

## Exploring Agent Steps and Messages

## Viewing the Full Conversation Context

## Chaining Agents for Creative Workflows

## Tracking the Final Agent Output